***
# <font color=red>Keyphrase Extraction using KeyBERT</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Team </font></p>

***

<font color=gray>ADS Sample Notebook.

Copyright (c) 2021 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Overview
This notebook demo will demonstrate a package called KeyBERT. KeyBERT makes it very simple to take any BERT model and do keyphrase extraction with it. KeyPhrase extraction has a wide variety of use-cases and is a canonical use-case for state of the art NLP techniques like transformer architecture neural networks. 

We specifically showcase a domain specific Keyphrase extraction model called [SPECTER](https://arxiv.org/pdf/2004.07180.pdf) which enables for effective Keyphrase extraction of academic texts. 

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

## Prerequisites:
 - Experience with the topic: Intermediate
 - Professional experience: Some
 
This notebook is intended for Data Scientists with experience doing Natural Language Processing tasks. We advice users to familiarize themselves with the recent advances within NLP as a result of the Transformer neural network architecture before running this notebook.

---

### First, import the necessary libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

from keybert import KeyBERT

Keybert is extremely simple to use. Give it a text string and initialize the model with the architecture and weights combination that you want. Other pre-trained models can be found here: https://www.sbert.net/docs/pretrained_models.html

Additionally, a user could train their own BERT model and directly load it from disk, rather than using a pre-trained model downloaded from the internet.

KeyBERT works by using documented embeddings generated with sentence transformer models. Then, embeddings are extracted for the N-gram words/phrases. Finally, cosine similarity scores are used to evaluate the similarity of the keywords to the document vector. This identifies the most similar words or phrases to the entire document. 

We use the pre-trained SPECTER model provided by AllenAI. It is highly effective for keyword mining of academic texts. It was trained on the SemanticScholar corpus.

In [ ]:
doc = """
Model explanation makes it easier to understand why machine learning models return the results that they do by identifying relative importance of features and relationships between features and predictions. Data Science offers the first commercial implementation of model-agnostic explanation. For example, a compliance officer can be certain that a model is not making decisions in violation of GDPR or regulations against discrimination.
For data scientists, it enables them to ensure that any model they build is generating results based on predictors that make sense. Understanding why a model behaves the way it does is critical to users and regulators. Data Science ensures that deployed models are more accurate, robust, and compliant with relevant regulations.
Oracle provides Machine Learning Explainablility (MLX), which is a package that explains the internal mechanics of a machine learning system to better understand models. Models are in the ADSModel format. You use MLX to explain models from different training platforms. You create an ADSModel from a REST end point then use the ADS model explainability to explain a model that’s remote.
      """
model = KeyBERT("allenai-specter")
keywords = model.extract_keywords(doc)

KeyBERT's `model.extract_keywords()` returns a list of tuples corresponding to each keyword and its cosine similarity to the document

In [ ]:
keywords

By default, a list of stop words is used which prevents common words like "the" or "and" from being included. You can turn this off by passing `stop_words = None` to the model

In [ ]:
model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words=None)

The range of generate ngrams can be specified with the `keyphrase_ngram_range` paramater. Usually, longer keyphrases have better semantic similarity scores than shorter ones.

In [ ]:
model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None)

Another approach besides the more traditional one can be used to get more diverse results. This is called the "Max Sum Similarity" algorithim. It is implemented by KeyBERT like this:

You can take the 2 x top_n most similar words/phrases to the document. Then, take all top_n combinations from the 2 x top_n words and extract the combination that are the least similar to each other by cosine similarity. T

this is enabled within KeyBERT by passing the `extract_keywords()` method the `max_sum=True` argument. One can specify `top_n` in a similar way

In [ ]:
model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', 
                           use_maxsum=True, nr_candidates=20, top_n=5)

Another way to diversify the results is to use the "Maximimal Marginal Relevance" statistic for weighting. The MMR method computes similarity scores between a proposed keyphrase and both the document *and* already selected keyphrases. It reranks keyphrases which are similar to the document based on how disimilar they are to each other based on the provided `diversity` creiterion.

In [ ]:
model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)

We can reduce the diversity and observe that the key phrases become more similar to each other

In [ ]:
model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.2)

We can compare the performance of this domain specific model with a more generic model

In [ ]:
model2 = KeyBERT('distilbert-base-nli-mean-tokens')
keywords2 = model2.extract_keywords(doc)

In [ ]:
keywords2

In [ ]:
model2.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)